In [1]:
# Simulating Game Result, and Team Composition based on the full game data provided

# This model takes in all features as inputs except a few features which it will predict in the end.
# The specific features it will predict are: ['result', 'gamelength', 'champion',
#'kills', 'deaths', 'assists', 'doublekills', 'triplekills', 'quadrakills', 
#'pentakills', 'firstblood', 'barons', 'opp_barons', 'inhibitors', 'opp_inhibitors', 'total cs']

# The prediction of this model is heavily influenced by the data provided.

# If you want to see the data which it uses when prediciting or to tweak it:
#   League Of Legends Esports Analysis - Group 1\Data\PredictionData\2023WorldsPredictions(ModelB).csv

# If you want to see the original data that is not modified then you can read the following
#   League Of Legends Esports Analysis - Group 1\Data\PredictionData\2023WorldsPredictions(ORIGINALDATA)

# Also MAKE SURE TO ADD IN YOUR ABSOLUTE PATH BEFORE RUNNING IN 'df_files'. Please CRTL+F and replace
# with your own abolute path to this project directory.

In [2]:
# Make Notebook cells strech to fit the browsers's full width resolution
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# Importing Libraries
import pandas as pd
import numpy as np
#import tensorflow as tf
import os

from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from matplotlib import pyplot as plt
from sklearn.metrics import log_loss
from sklearn.metrics import accuracy_score
from glob import glob
from pathlib import Path
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_hastie_10_2
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.datasets import load_iris
from sklearn.preprocessing import OrdinalEncoder

from sklearn.tree import DecisionTreeRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_squared_error
#from keras.utils.vis_utils import plot_model
import re

# Grab all files
# To run this add in the absolute path to the /Data/2023.csv
df_files = sorted(glob("/Users/Gagan/Documents/Anaconda Projects/League Of Legends Esports Analysis - Group 1/Data/2023.csv")) # 20**.csv

# Combine all the files into one big dataframe
df_2022 = pd.concat((pd.read_csv(file, low_memory=False) for file in df_files))
print(df_files)


# Filter rows based on data completeness and position
df_2022 = df_2022[(df_2022['datacompleteness'] == 'complete') & (df_2022['position'] != 'team')]

# Drop unnecessary columns
cols_to_drop = ['datacompleteness', 'url', 'split', 'playoffs', 'date', 'game','participantid', 'playerid', 'teamid', 'firstbloodassist', 'firstbloodkill', 'firstbloodvictim', 'dragons (type unknown)', 'damageshare', 'earnedgoldshare', 'monsterkillsownjungle', 'monsterkillsenemyjungle', 'year', 'patch', 'teamkills', 'teamdeaths']
df_2022 = df_2022.drop(columns=cols_to_drop)
df_2022.fillna(0, inplace=True)

# Fill NaN values in specific columns
df_2022.fillna({'ban1': 'UNKNOWN', 'ban2': 'UNKNOWN', 'ban3': 'UNKNOWN', 'ban4': 'UNKNOWN', 'ban5': 'UNKNOWN', 'firstmidtower': 0}, inplace=True)

champions = pd.concat([df_2022['ban1'], df_2022['ban2'], df_2022['ban3'], df_2022['ban4'], df_2022['ban5']]).unique()
allChampions = df_2022['champion'].unique()
allLeagues = df_2022['league'].unique()
allPlayerNames = df_2022['playername'].unique()
allTeamNames = df_2022['teamname'].unique()

# Convert categorical values to numerical values
value_to_index = {value: index for index, value in enumerate(champions)}
value_to_index1 = {value: index for index, value in enumerate(allChampions)}
value_to_index2League = {value: index for index, value in enumerate(allLeagues)}
value_to_index3PlayerNames = {value: index for index, value in enumerate(allPlayerNames)}
value_to_index4TeamNames = {value: index for index, value in enumerate(allTeamNames)}

df_2022['teamname'] = df_2022['teamname'].map(value_to_index4TeamNames)
df_2022['playername'] = df_2022['playername'].map(value_to_index3PlayerNames)
df_2022['champion'] = df_2022['champion'].map(value_to_index1)
df_2022['league'] = df_2022['league'].map(value_to_index2League)
df_2022['ban1'] = df_2022['ban1'].map(value_to_index)
df_2022['ban2'] = df_2022['ban2'].map(value_to_index)
df_2022['ban3'] = df_2022['ban3'].map(value_to_index)
df_2022['ban4'] = df_2022['ban4'].map(value_to_index)
df_2022['ban5'] = df_2022['ban5'].map(value_to_index)

df_2022['side'] = df_2022['side'].map({'Blue': 0, 'Red': 1})

df_2022['position'] = df_2022['position'].map({'top': 0, 'jng': 1, 'mid': 2, 'bot': 3, 'sup': 4})

extracted_data = []
for game_id in df_2022['gameid']:
    if pd.notna(game_id):
        if '/' in game_id:
            extracted_data.append(game_id.split('/')[-1])
        elif '-' in game_id:
            extracted_data.append(game_id.replace('-', '')[1])

        elif '_' in game_id:
            if '_game_' in game_id:
                extracted_data.append(game_id.split('_game_')[1])
            else:
                extracted_data.append(game_id.replace('-', '').split('_')[1])
        else:
            extracted_data.append(game_id)
    else:
        extracted_data.append(np.nan)
        
df_2022['gameid'] = extracted_data       
#print(df_2022['gameid'] )

X = df_2022.drop(columns=['result', 'gamelength', 'champion', 'kills', 'deaths', 'assists', 'doublekills', 
                          'triplekills', 'quadrakills', 'pentakills', 'firstblood', 'barons', 'opp_barons',
                          'inhibitors', 'opp_inhibitors', 'total cs'])#(columns=[14])

y = df_2022[['result', 'gamelength', 'champion', 'kills', 'deaths', 'assists', 'doublekills', 
             'triplekills', 'quadrakills', 'pentakills', 'firstblood', 'barons', 'opp_barons',
             'inhibitors', 'opp_inhibitors', 'total cs']]#[14]

columns_to_exclude_scaling = ['side', 'ban1', 'ban2', 'ban3', 'ban4', 'ban5']
columns_to_scale = df_2022.columns.difference(columns_to_exclude_scaling)
scaler = MinMaxScaler()
df_2022[columns_to_scale] = scaler.fit_transform(df_2022[columns_to_scale])

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

#model = HistGradientBoostingClassifier()
base_regressor = DecisionTreeRegressor()

# Wrap the base regressor in MultiOutputRegressor
model = MultiOutputRegressor(base_regressor)

model.fit(X, y)
# Get feature importances
feature_importance = model.score(X, y)

# To run this add in the absolute path to the /Data/PredictionData/2023WorldsPredictions(ModelB).csv
df_files = "C:/Users/Gagan/Documents/Anaconda Projects/League Of Legends Esports Analysis - Group 1/Data/PredictionData/2023WorldsPredictions(ModelB).csv"
print(df_files)

df_2023Predictions = pd.read_csv(df_files, low_memory=False)

#print(df_2022.dtypes)
#print(df_2022.head)


Xprediction = df_2023Predictions.drop(columns=['champion', 'gamelength', 'barons', 'opp_barons', 'inhibitors', 'opp_inhibitors', 'total cs', 
                                                'kills', 'deaths', 'assists', 'doublekills', 'triplekills', 'quadrakills', 'pentakills', 'firstblood',
                                               'datacompleteness', 'url', 'split', 'playoffs', 'date', 'game','participantid', 'playerid', 'teamid',
                                               'firstbloodassist', 'firstbloodkill', 'firstbloodvictim', 'dragons (type unknown)', 'damageshare', 
                                               'earnedgoldshare', 'monsterkillsownjungle', 'monsterkillsenemyjungle', 'year', 'patch', 'teamkills', 'teamdeaths', 'result'])

#print(df_2023Predictions.dtypes)

#print(feature_importance)


# Combine all the files into one big dataframe

champions = pd.concat([df_2022['ban1'], df_2022['ban2'], df_2022['ban3'], df_2022['ban4'], df_2022['ban5']]).unique()

# Convert categorical values to numerical values
value_to_index = {value: index for index, value in enumerate(champions)}


# Convert categorical values to numerical values
#value_to_index1 = {value: index for index, value in enumerate(allChampions)} 
value_to_index2League = {value: index for index, value in enumerate(allLeagues)}
value_to_index3PlayerNames = {value: index for index, value in enumerate(allPlayerNames)}
value_to_index4TeamNames = {value: index for index, value in enumerate(allTeamNames)}

Xprediction['teamname'] = Xprediction['teamname'].map(value_to_index4TeamNames)
Xprediction['playername'] = Xprediction['playername'].map(value_to_index3PlayerNames)
#Xprediction['champion'] = Xprediction['champion'].map(value_to_index1) 
Xprediction['league'] = Xprediction['league'].map(value_to_index2League)

Xprediction['side'] = Xprediction['side'].map({'Blue': 0, 'Red': 1})

Xprediction['position'] = Xprediction['position'].map({'top': 0, 'jng': 1, 'mid': 2, 'bot': 3, 'sup': 4})

Xprediction['gameid'] = Xprediction['gameid'].str.extract(r'_(\d+)$')

Xprediction['ban1'] = Xprediction['ban1'].map(value_to_index)
Xprediction['ban2'] = Xprediction['ban2'].map(value_to_index)
Xprediction['ban3'] = Xprediction['ban3'].map(value_to_index)
Xprediction['ban4'] = Xprediction['ban4'].map(value_to_index)
Xprediction['ban5'] = Xprediction['ban5'].map(value_to_index)

#print(Xprediction.shape)
predictions = model.predict(Xprediction)

y = df_2022[['result', 'gamelength', 'champion', 'kills', 'deaths', 'assists', 'doublekills', 
                          'triplekills', 'quadrakills', 'pentakills', 'firstblood', 'barons', 'opp_barons',
                          'inhibitors', 'opp_inhibitors', 'total cs']]
predicted_df = pd.DataFrame(predictions, columns=y.columns)
pd.set_option('display.max_columns', None)
#print(allChampions)
value_to_index1 = {value: index for index, value in enumerate(allChampions)} 
index_to_champion = {index: value for value, index in value_to_index1.items()}
predicted_df['champion'] = predicted_df['champion'].map(index_to_champion)

predicted_df = pd.DataFrame(predicted_df)

#print(predicted_df)
#result_df = pd.merge(predicted_df, df_2023Predictions, left_index=True, right_index=True, how='left')
df_2023PredClean = pd.read_csv(df_files, low_memory=False)
df_2023PredClean = df_2023PredClean.drop(columns=cols_to_drop)

# Update values in result_df with values from predicted_df where not null
merged_df = pd.merge(left=df_2023PredClean, right=predicted_df, how='left', left_index = True, right_index = True)
merged_df = merged_df.drop(columns=[col for col in merged_df.columns if '_x' in col])
merged_df = merged_df.rename(columns=lambda x: x.split('_y')[0] if '_y' in x else x)

display(merged_df)


['/Users/Gagan/Documents/Anaconda Projects/League Legends Win predictor/League Legends Win predictor/Data/2023.csv']
C:/Users/Gagan/Documents/Anaconda Projects/League Legends Win predictor/League Legends Win predictor/Data/PredictionData/2023WorldsPredictions(ModelB).csv


,gameid,league,side,position,playername,teamname,ban1,ban2,ban3,ban4,ban5,team kpm,ckpm,firstdragon,dragons,opp_dragons,elementaldrakes,opp_elementaldrakes,infernals,mountains,clouds,oceans,chemtechs,hextechs,elders,opp_elders,firstherald,heralds,opp_heralds,firstbaron,firsttower,towers,opp_towers,firstmidtower,firsttothreetowers,turretplates,opp_turretplates,damagetochampions,dpm,damagetakenperminute,damagemitigatedperminute,wardsplaced,wpm,wardskilled,wcpm,controlwardsbought,visionscore,vspm,totalgold,earnedgold,earned gpm,goldspent,gspd,minionkills,monsterkills,cspm,goldat10,xpat10,csat10,opp_goldat10,opp_csat10,golddiffat10,xpdiffat10,csdiffat10,killsat10,assistsat10,deathsat10,opp_killsat10,opp_assistsat10,opp_deathsat10,goldat15,xpat15,csat15,opp_goldat15,opp_csat15,golddiffat15,xpdiffat15,csdiffat15,killsat15,assistsat15,deathsat15,opp_killsat15,opp_assistsat15,opp_deathsat15,result,gamelength,champion,kills,deaths,assists,doublekills,triplekills,quadrakills,pentakills,firstblood,barons,opp_barons,inhibitors,opp_inhibitors,total cs
0,ESPORTSTMNT02_4000000,WLDs,Red,top,TheShy,Weibo Gaming,Maokai,Varus,Gragas,Alistar,Rell,0.3401,0.8219,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25967,735.9565,1016.6840,1121.5777,17,0.4818,7,0.1984,4,46,1.3037,13417,8823,250.0614,12833,NaN,234,7,6.8304,3101,4875,81,3215,83,-114,132,-2,0,0,0,0,0,0,5408,7742,127,6305,143,-897,-539,-16,1,0,0,1,1,0,0.0,2117.0,Renekton,5.0,4.0,6.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,241.0
1,ESPORTSTMNT02_4000000,WLDs,Red,jng,Weiwei,Weibo Gaming,Maokai,Varus,Gragas,Alistar,Rell,0.3401,0.8219,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10086,285.8573,973.6042,1742.1256,6,0.1701,17,0.4818,5,45,1.2754,10265,5671,160.7274,10075,NaN,37,153,5.3850,3117,2900,61,3699,61,-582,-511,0,0,2,2,1,0,0,4495,4627,87,5472,79,-977,-75,8,0,3,3,3,1,0,0.0,2117.0,Rell,0.0,5.0,10.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,190.0
2,ESPORTSTMNT02_4000000,WLDs,Red,mid,Xiaohu,Weibo Gaming,Maokai,Varus,Gragas,Alistar,Rell,0.3401,0.8219,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19716,558.7907,497.9121,333.5002,16,0.4535,3,0.0850,5,32,0.9069,11617,7023,199.0458,10275,NaN,206,16,6.2919,3821,4732,82,4818,98,-997,-617,-16,2,0,1,2,0,0,5242,6737,121,7227,145,-1985,-697,-24,2,0,2,3,1,1,0.0,2117.0,Taliyah,4.0,3.0,3.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,222.0
3,ESPORTSTMNT02_4000000,WLDs,Red,bot,Light,Weibo Gaming,Maokai,Varus,Gragas,Alistar,Rell,0.3401,0.8219,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,26606,754.0671,559.4143,388.3987,9,0.2551,7,0.1984,0,24,0.6802,13993,9399,266.3864,13650,NaN,306,12,9.0128,3375,2527,63,3101,65,274,-590,-2,1,1,1,0,2,1,5055,4983,115,5110,129,-55,-1068,-14,1,1,2,0,2,1,0.0,2117.0,Jinx,3.0,3.0,6.0,0.0,0.0,0.0,0.0,1.0,0.0,2.0,0.0,1.0,318.0
4,ESPORTSTMNT02_4000000,WLDs,Red,sup,Crisp,Weibo Gaming,Maokai,Varus,Gragas,Alistar,Rell,0.3401,0.8219,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3775,106.9910,470.1653,232.0076,46,1.3037,6,0.1701,13,80,2.2674,7414,2820,79.9244,7210,NaN,33,0,0.9353,2436,2417,14,2930,11,-494,223,3,0,3,0,1,2,2,3444,3540,24,4432,12,-988,-43,12,0,3,1,1,6,2,0.0,2117.0,Milio,0.0,3.0,8.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,33.0
5,ESPORTSTMNT02_4000000,WLDs,Blue,top,Zeus,LNG Esports,Akali,Syndra,Ahri,Xayah,Sylas,0.4818,0.8219,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16081,455.7676,1189.6552,1530.6944,5,0.1417,7,0.1984,2,27,0.7652,13094,8500,240.9069,13280,NaN,264,13,7.8507,3215,4743,83,3101,81,114,-132,2,0,0,0,0,0,0,6305,8281,143,5408,127,897,539,16,1,1,0,1,0,0,1.0,2139.0,Cassiopeia,1.0,3.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,277.0
6,ESPORTSTMNT02_4000000,WLDs,Blue,jng,Oner,LNG Esports,Akali,Syndra,Ahri,Xayah,Sylas,0.4818,0.8219,NaN,NaN,NaN,NaN,NaN,